In [1]:
import pandas as pd
import string

In [2]:
# Abre o arquivo em modo de leitura
with open("todas-palavras-pt-br.txt", "r", encoding="utf-8") as file:
    # Lê todas as linhas do arquivo e as une em uma única string
    texto_completo = " ".join(file.readlines())
    # Divide a string em uma lista de palavras
    palavras = texto_completo.split()

# Cria um DataFrame do Pandas com uma coluna chamada "palavra"
df = pd.DataFrame({"palavra": palavras})
df

,palavra
0,a
1,Aarao
2,aba
3,abacate
4,abacateiro
...,...
245361,zurremos
245362,zurres
245363,zurro
245364,zurros


In [3]:
# Cria um novo DataFrame apenas com palavras de cinco letras
df_5_letras = df[df["palavra"].str.len() == 5]

# Redefine o índice do novo DataFrame
df_5_letras = df_5_letras.reset_index(drop=True)

# transforma todas letras em minuscula
df_5_letras["palavra"] = df_5_letras["palavra"].str.lower()
df_5_letras

# Cria as colunas l1, l2, l3, l4 e l5
df_5_letras[['l1', 'l2', 'l3', 'l4', 'l5']] = df_5_letras['palavra'].str.pad(5, side='right', fillchar='_').str.extract(r'(.)(.)(.)(.)(.)')
df_5_letras

,palavra,l1,l2,l3,l4,l5
0,aarao,a,a,r,a,o
1,abaco,a,b,a,c,o
2,abade,a,b,a,d,e
3,abafa,a,b,a,f,a
4,abafe,a,b,a,f,e
...,...,...,...,...,...,...
5476,zunis,z,u,n,i,s
5477,zuniu,z,u,n,i,u
5478,zurra,z,u,r,r,a
5479,zurre,z,u,r,r,e


In [4]:
# Função que calcula a força de cada palavra, descontabilizando letras repetidas
def calcular_forca_total(linha):
    letras_usadas = set()
    forca_total = 0
    for i in range(1, 6):
        letra = linha[f"l{i}"]
        if letra not in letras_usadas:
            forca_total += linha[f"f{i}"]
            letras_usadas.add(letra)
    return forca_total

# Função que calcula a força de todas palavras no df
def calculo_forca_df(df):
    # Cria uma lista com todas as letras do alfabeto
    alfabeto = list(string.ascii_lowercase)

    # Cria o DataFrame auxiliar
    df_alfabeto = pd.DataFrame({"letra": alfabeto})
    
    # Cria a coluna "numero_ocorrencias" no df_alfabeto e inicializa com 0
    df_alfabeto["numero_ocorrencias"] = 0

    # Itera sobre cada letra no df_alfabeto
    for indice, linha in df_alfabeto.iterrows():
        letra = linha["letra"]

        # Conta quantas palavras no df contêm a letra
        contagem = df["palavra"].str.contains(letra).sum()

        # Atualiza o df_alfabeto com a contagem
        df_alfabeto.loc[indice, "numero_ocorrencias"] = contagem

    # calculando força de cada letra
    for i in range(1, 6):
        coluna_letra = f"l{i}"
        coluna_forca = f"f{i}"

        # Cria um mapeamento entre letras e suas forças
        mapeamento_forca = dict(zip(df_alfabeto["letra"], df_alfabeto["numero_ocorrencias"]))

        # Aplica o mapeamento à coluna de letras correspondente
        df[coluna_forca] = df[coluna_letra].map(mapeamento_forca)
    
    df["f_total"] = df.apply(calcular_forca_total, axis=1)
    
    return df

In [5]:
# Função que imprime os melhores palpites para o df selecionado
def melhores_palpites(df):
    numero_palavras = len(df)
    numero_top = min(5, numero_palavras)  # Mostra até 5, ou o número de palavras disponíveis
    top_palavras = df.nlargest(numero_top, "f_total")["palavra"].tolist()

    print(f"Existem {numero_palavras} palavras possíveis.\n"
          "Os melhores palpites são:")
    
    for i in range(numero_top):
        print(f"{i+1}. {top_palavras[i]}")


In [6]:
def filtrar_df(df, chute, o1, o2, o3, o4, o5):
    for i, (letra, cor) in enumerate(zip(chute, [o1, o2, o3, o4, o5])):
        if cor == 'v':
            # Verde: mantém palavras com a letra na posição exata
            df = df[df[f"l{i+1}"] == letra]
        elif cor == 'a':
            # Amarelo: mantém palavras com a letra em outra posição
            df = df[df["palavra"].str.contains(letra)]
            df = df[df[f"l{i+1}"] != letra]
        elif cor == 'p':
            # Preto: remove palavras com a letra
            df = df[~df["palavra"].str.contains(letra)]
    return df


In [10]:
def main():
    df = df_5_letras.copy()  # Crie uma cópia para não modificar o DataFrame original

    while True:
        df = calculo_forca_df(df)
        melhores_palpites(df)
        print("")

        while True:  # Loop para validar o chute
            chute = input("Insira palavra chutada (5 letras): ").lower()
            if len(chute) == 5:
                break
            else:
                print("Palavra inválida! Insira uma palavra com 5 letras.")

        print("")
        print("Escreva 'v' para verde, 'a' para amarelo e 'p' para preto.")

        # Loop para validar as cores
        while True:
            o1 = input("Letra 1 - " + chute[0] + ": ").lower()
            o2 = input("Letra 2 - " + chute[1] + ": ").lower()
            o3 = input("Letra 3 - " + chute[2] + ": ").lower()
            o4 = input("Letra 4 - " + chute[3] + ": ").lower()
            o5 = input("Letra 5 - " + chute[4] + ": ").lower()

            if all(c in 'vap' for c in [o1, o2, o3, o4, o5]):
                break
            else:
                print("Entrada inválida! Use apenas 'v', 'a' ou 'p'.")

        print("")

        if o1 + o2 + o3 + o4 + o5 == "vvvvv":
            print("Parabéns, você ganhou!")
            break

        df = filtrar_df(df.copy(), chute, o1, o2, o3, o4, o5)  # Filtra o DataFrame

        if len(df) == 0:
            print("Não há mais palavras possíveis. Algo deu errado!")
            break

In [12]:
main()

Existem 5481 palavras possíveis.
Os melhores palpites são:
1. rosea
2. serao
3. orais
4. raios
5. roais

Insira palavra chutada (5 letras): serao

Escreva 'v' para verde, 'a' para amarelo e 'p' para preto.
Letra 1 - s: p
Letra 2 - e: p
Letra 3 - r: p
Letra 4 - a: p
Letra 5 - o: v

Existem 173 palavras possíveis.
Os melhores palpites são:
1. unico
2. lucio
3. intuo
4. unimo
5. lindo

Insira palavra chutada (5 letras): unico

Escreva 'v' para verde, 'a' para amarelo e 'p' para preto.
Letra 1 - u: a
Letra 2 - n: p
Letra 3 - i: p
Letra 4 - c: p
Letra 5 - o: v

Existem 17 palavras possíveis.
Os melhores palpites são:
1. multo
2. vulto
3. dublo
4. vulgo
5. duplo

Insira palavra chutada (5 letras): vulto

Escreva 'v' para verde, 'a' para amarelo e 'p' para preto.
Letra 1 - v: p
Letra 2 - u: v
Letra 3 - l: a
Letra 4 - t: p
Letra 5 - o: v

Existem 2 palavras possíveis.
Os melhores palpites são:
1. dublo
2. duplo

Insira palavra chutada (5 letras): duplo

Escreva 'v' para verde, 'a' para amarelo